In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [9]:
# Загрузка данных
df = pd.read_excel('повт_обработка.xlsx')  

In [75]:
import math

# Размер чанка (максимальное количество строк в каждом датафрейме)
CHUNK_SIZE = 3000

# Вычисляем количество чанков
num_chunks = math.ceil(len(df
                          ) / CHUNK_SIZE)

In [77]:
num_chunks

4

In [79]:
# Создаем словарь для хранения чанков
chunked_dfs = {}

In [81]:
# Разбиваем датафрейм на части
for i in range(num_chunks):
    start_idx = i * CHUNK_SIZE
    end_idx = (i + 1) * CHUNK_SIZE
    chunk_name = f'chunk_{i+1}'
    chunked_dfs[chunk_name] = df.iloc[start_idx:end_idx].copy()

# Выводим информацию о созданных датафреймах
print(f"Исходный датафрейм содержит {len(df)} уникальных организаций")
print(f"Разбито на {num_chunks} чанков по {CHUNK_SIZE} записей:")

Исходный датафрейм содержит 11905 уникальных организаций
Разбито на 4 чанков по 3000 записей:


In [83]:
from dadata import Dadata
TOKEN = "1b31bbb703a9f56724d07df47b61e2b905bcb976"
SECRET = "3daf5d980c9af80d387066f283f7c9b0eefe2d17"
dadata = Dadata(TOKEN, SECRET)

In [84]:
import pandas as pd
import time
import os

def enhanced_clean_address(address_str):
    """Улучшенная очистка адреса с гарантированным получением всех полей"""
    try:
        if pd.isna(address_str) or not address_str.strip():
            return None
            
        # Пытаемся сначала найти через suggest (бесплатно)
        suggest_result = dadata.suggest("address", address_str, count=1)
        if suggest_result:
            data = suggest_result[0]['data']
            return {
                'result': suggest_result[0]['value'],
                'postal_code': data.get('postal_code'),
                'federal_district': data.get('federal_district'),
                'region': data.get('region_with_type'),
                'area': data.get('area_with_type'),
                'city': data.get('city_with_type'),
                'street': data.get('street_with_type'),
                'house': data.get('house'),
                'geo_lat': data.get('geo_lat'),
                'geo_lon': data.get('geo_lon'),
                'source': 'suggest'
            }
        
        # Если через suggest не получилось, пробуем clean (может быть платным)
        cleaned = dadata.clean("address", address_str)
        if cleaned:
            return {
                'result': cleaned.get('result'),
                'postal_code': cleaned.get('postal_code'),
                'federal_district': cleaned.get('federal_district'),
                'region': cleaned.get('region_with_type'),
                'area': cleaned.get('area_with_type'),
                'city': cleaned.get('city_with_type'),
                'street': cleaned.get('street_with_type'),
                'house': cleaned.get('house'),
                'geo_lat': cleaned.get('geo_lat'),
                'geo_lon': cleaned.get('geo_lon'),
                'source': 'clean'
            }
    
    except Exception as e:
        print(f"Ошибка при обработке адреса {address_str}: {e}")
    
    return None

In [85]:
def process_address_without_zip(address_str):
    """Удаляем почтовый индекс из адреса"""
    if pd.isna(address_str) or not address_str.strip():
        return address_str
    
    # Убираем почтовый индекс (6 цифр в начале)
    import re
    cleaned_address = re.sub(r'^\d{6},\s*', '', str(address_str))
    return cleaned_address.strip()

In [89]:
def process_data_safely(df, chunk_size=3, delay=2):
    """Безопасная обработка с контролем лимитов"""
    results = []
    total = len(df)
    
    for idx, row in df.iterrows():
        # Берем адрес только из столбца adr
        address = str(row['adr']) if pd.notna(row['adr']) else None
        
        # Формируем базовый результат
        result = {
            'source_address': address,
            'found_by': None,
            'result': None,
            'postal_code': None,
            'federal_district': None,
            'region': None,
            'area': None,
            'city': None,
            'street': None,
            'house': None,
            'geo_lat': None,
            'geo_lon': None,
            'source': None
        }
        
        # Чистим адрес
        if address:
        # Очищаем адрес от почтового индекса
            clean_address = process_address_without_zip(address)
            cleaned = enhanced_clean_address(clean_address)
        # if address:
        #     cleaned = enhanced_clean_address(address)
            if cleaned:
                result.update({
                    'found_by': 'адрес',
                    'result': cleaned.get('result'),
                    'postal_code': cleaned.get('postal_code'),
                    'federal_district': cleaned.get('federal_district'),
                    'region': cleaned.get('region'),
                    'area': cleaned.get('area'),
                    'city': cleaned.get('city'),
                    'street': cleaned.get('street'),
                    'house': cleaned.get('house'),
                    'geo_lat': cleaned.get('geo_lat'),
                    'geo_lon': cleaned.get('geo_lon'),
                    'source': cleaned.get('source')
                })
        
        results.append(result)
        
        # Соблюдаем лимиты
        if (idx + 1) % chunk_size == 0 and (idx + 1) < total:
            print(f"Обработано {idx+1}/{total}. Пауза {delay} сек...")
            time.sleep(delay)
    
    return pd.DataFrame(results)


In [91]:
# Запуск обработки
try:
    print("Начало обработки данных...")
    result_df = process_data_safely(
        chunked_dfs['chunk_1'],
        chunk_size=3,  # 3 запроса подряд
        delay=3        # затем пауза 3 секунды
    )
    
    # Создаем копию исходного DataFrame для слияния
    chunk_df = chunked_dfs['chunk_1'].copy()
    
    # Просто объединяем по индексу, так как порядок сохранен
    result_df_ad = pd.concat([chunk_df.reset_index(drop=True), result_df.reset_index(drop=True)], axis=1)
    
    # Альтернативный вариант - слияние по адресу
    # result_df_ad = pd.merge(
    #     chunk_df,
    #     result_df,
    #     left_on='adr',
    #     right_on='source_address',
    #     how='left'
    # )
    
    # Определяем путь для сохранения
    output_folder = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3\processing3" 
    output_filename = "повт_обработанные.csv"  
    output_path = os.path.join(output_folder, output_filename)

    # Проверяем и создаем папку, если ее нет
    os.makedirs(output_folder, exist_ok=True)

    # Сохранение результатов
    result_df_ad.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты сохранены в {output_path}")
    
    # Показать первые несколько строк для проверки
    print("\nПервые 5 строк результата:")
    print(result_df_ad.head())
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Обработано 3/3000. Пауза 3 сек...
Ошибка при обработке адреса Московская область, г. Химки, Химки-Центр, ул. Панфилова, вл. 19, стр. 1: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Обработано 6/3000. Пауза 3 сек...
Ошибка при обработке адреса г. Новосибирск, ул. М. Горького, д. 53: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса г. Самара, ул. Чапаевская/Красноармейская, д.178/12: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Обработано 9/3000. Пауза 3 сек...
Ошибка при обработке адреса г. Челябинск, ул. К. Маркса, д. 38: Client error '403 ' for url 'https://cleaner.dadata.ru/api

In [95]:
# Запуск обработки chunk_2
try:
    print("Начало обработки данных chunk_2...")
    result_df2 = process_data_safely(
        chunked_dfs['chunk_2'],
        chunk_size=3,  # 3 запроса подряд
        delay=3        # затем пауза 3 секунды
    )
    
    # Создаем копию исходного DataFrame для слияния
    chunk_df2 = chunked_dfs['chunk_2'].copy()
    
    # Добавляем временный индексный столбец для слияния
    chunk_df2['temp_index'] = range(len(chunk_df2))
    result_df2['temp_index'] = range(len(result_df2))
    
    # Сливаем по временному индексу
    result_df_ad2 = pd.merge(
        chunk_df2,
        result_df2,
        on='temp_index',
        how='left'
    )
    
    # Удаляем временный столбец
    result_df_ad2 = result_df_ad2.drop('temp_index', axis=1)
    
    # Проверяем, существует ли уже файл с результатами
    if os.path.exists(output_path):
        # Читаем существующие данные
        existing_df = pd.read_csv(output_path)
        # Объединяем с новыми результатами
        combined_df = pd.concat([existing_df, result_df_ad2], ignore_index=True)
    else:
        combined_df = result_df_ad2
        
    # Сохранение результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_2 добавлены к {output_path}")
    
    # Показать первые несколько строк для проверки
    print("\nПервые 5 строк результата chunk_2:")
    print(result_df_ad2.head())
        
except Exception as e:
    print(f"Ошибка обработки chunk_2: {e}")
    import traceback
    traceback.print_exc()

Начало обработки данных chunk_2...
Ошибка при обработке адреса Белгородская область, Прохоровский район, с. Журавка-1, ул. Административная, д. 2: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Белгородская обл., Новооскольский р-он, с. Великомихайловка, пл. 1-й Конной Армии, 1: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Белгородская область, пгт. Чернянка ул. Октябрьская, 1а: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Белгородская область, пгт. Волоконовка, ул. Ленина, 18: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/addr

In [97]:
# Запуск обработки chunk_3
try:
    print("Начало обработки данных chunk_3...")
    result_df3 = process_data_safely(
        chunked_dfs['chunk_3'],
        chunk_size=3,  # 3 запроса подряд
        delay=3        # затем пауза 3 секунды
    )
    
    # Создаем копию исходного DataFrame для слияния
    chunk_df3 = chunked_dfs['chunk_3'].copy()
    
    # Добавляем временный индексный столбец для слияния
    chunk_df3['temp_index'] = range(len(chunk_df3))
    result_df3['temp_index'] = range(len(result_df3))
    
    # Сливаем по временному индексу
    result_df_ad3 = pd.merge(
        chunk_df3,
        result_df3,
        on='temp_index',
        how='left'
    )
    
    # Удаляем временный столбец
    result_df_ad3 = result_df_ad3.drop('temp_index', axis=1)
    
    # Проверяем, существует ли уже файл с результатами
    if os.path.exists(output_path):
        # Читаем существующие данные
        existing_df = pd.read_csv(output_path)
        # Объединяем с новыми результатами
        combined_df = pd.concat([existing_df, result_df_ad3], ignore_index=True)
    else:
        combined_df = result_df_ad3
        
    # Сохранение результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_3 добавлены к {output_path}")
    
    # Показать первые несколько строк для проверки
    print("\nПервые 5 строк результата chunk_3:")
    print(result_df_ad3.head())
        
except Exception as e:
    print(f"Ошибка обработки chunk_3: {e}")
    import traceback
    traceback.print_exc()

Начало обработки данных chunk_3...
Ошибка при обработке адреса Смоленская обл., Дорогобужский р-н, пос. Верхнеднепровский, ул. Молодежная, д. 10: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Смоленская область, г.Сычёвка, ул. К.Маркса,д.1: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Ставропольский край, г. Ставрополь, ул. Черняховского, 8: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Ставропольский край, г. Ставрополь, ул. Ленина, 381, в квартале 480: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more informatio

In [99]:
# Запуск обработки chunk_4
try:
    print("Начало обработки данных chunk_4...")
    result_df4 = process_data_safely(
        chunked_dfs['chunk_4'],
        chunk_size=3,  # 3 запроса подряд
        delay=3        # затем пауза 3 секунды
    )
    
    # Создаем копию исходного DataFrame для слияния
    chunk_df4 = chunked_dfs['chunk_4'].copy()
    
    # Добавляем временный индексный столбец для слияния
    chunk_df4['temp_index'] = range(len(chunk_df4))
    result_df4['temp_index'] = range(len(result_df4))
    
    # Сливаем по временному индексу
    result_df_ad4 = pd.merge(
        chunk_df4,
        result_df4,
        on='temp_index',
        how='left'
    )
    
    # Удаляем временный столбец
    result_df_ad4 = result_df_ad4.drop('temp_index', axis=1)
    
    # Проверяем, существует ли уже файл с результатами
    if os.path.exists(output_path):
        # Читаем существующие данные
        existing_df = pd.read_csv(output_path)
        # Объединяем с новыми результатами
        combined_df = pd.concat([existing_df, result_df_ad4], ignore_index=True)
    else:
        combined_df = result_df_ad4
        
    # Сохранение результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_4 добавлены к {output_path}")
    
    # Показать первые несколько строк для проверки
    print("\nПервые 5 строк результата chunk_4:")
    print(result_df_ad4.head())
        
except Exception as e:
    print(f"Ошибка обработки chunk_4: {e}")
    import traceback
    traceback.print_exc()

Начало обработки данных chunk_4...
Ошибка при обработке адреса Владимирская область, г. Ковров, ул. Чернышевского, д. 17, пом. IV: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Владимирская область, Собинский район, МО городское поселение Лакинск, ул. Мира, д.11а: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Владимирская область, г.Кольчугино, ул.3-го Интернационала, д.40: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Владимирская область, Петушинский район, г.Петушки, ул.Полевой проезд, д.4а: Client error '403 ' for url 'https://cleaner.dadata.ru/a

In [101]:
# Создаем копию датафрейма
new_df = combined_df.copy(deep=True)

In [107]:
# Сохранение в Excel с явным указанием формата ячеек (для корректного отображения номера дома)
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3\processing3"
excel_output_path = os.path.join(output_dir, f"повт_обработанные_итерация1_{timestamp}.xlsx")

with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
    new_df.to_excel(writer, index=False, sheet_name='Data')
    
    # Получаем лист и устанавливаем текстовый формат для столбца house
    worksheet = writer.sheets['Data']
    
    # Находим индекс столбца house
    house_col_idx = new_df.columns.get_loc('house') + 1
    
    # Устанавливаем текстовый формат для всего столбца
    for row in range(2, len(new_df) + 2):  # +2 потому что 1 строка - заголовок
        cell = worksheet.cell(row=row, column=house_col_idx)
        cell.number_format = '@'  # Текстовый формат

print(f"Excel файл успешно сохранен: {excel_output_path}")

Excel файл успешно сохранен: C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3\processing3\повт_обработанные_итерация1_20250820_152617.xlsx
